In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import scanpy as sc
import anndata as ad
import numpy as np
import pandas as pd
import anndata2ri
import gdown
import scipy
import scipy.io
from rpy2.robjects import r

anndata2ri.activate()

In [2]:
%load_ext rpy2.ipython

C:\Users\Chayan\anaconda3\lib\site-packages\rpy2\robjects\packages.py:366: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


In [4]:
%%R
suppressPackageStartupMessages({
    library(SingleCellExperiment)
    library(Seurat)
    library(SeuratDisk)
    library(dplyr)
})

## Importing the data

In [7]:
dir = 'E:/UMINTv2/Data/kotliarov2020/'

In [8]:
adata = sc.read(dir+'expressions_hvg.h5ad')
adata

AnnData object with n_obs × n_vars = 52117 × 3999
    obs: 'batch', 'cluster_level2', 'cluster_level3', 'sample', 'cell_type', 'n_genes', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt'
    var: 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'batch_colors', 'cell_type_colors', 'cluster_level2_colors', 'cluster_level3_colors', 'neighbors', 'pca', 'sample_colors', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    layers: 'count'
    obsp: 'connectivities', 'distances'

In [9]:
# Seurat categories have to be strings
adata.obs.batch = adata.obs.batch.cat.rename_categories(["1", "2"])

In [10]:
adata_ = ad.AnnData(adata.layers['count'])
adata_.obs_names = adata.obs_names
adata_.var_names = adata.var_names
adata_.obs['cell_type'] = adata.obs['cell_type']
adata_.obs['batch'] = adata.obs['batch']

In [11]:
%%R -i adata_
rna = as.Seurat(adata_, counts='X', data=NULL)
rna

An object of class Seurat 
3999 features across 52117 samples within 1 assay 
Active assay: originalexp (3999 features, 0 variable features)


In [12]:
adata = sc.read(dir+'protein.h5ad')

In [13]:
# Seurat categories have to be strings
adata.obs.batch = adata.obs.batch.cat.rename_categories(["1", "2"])

In [14]:
adata_ = ad.AnnData(adata.layers['count'])
adata_.obs_names = adata.obs_names
adata_.var_names = adata.var_names
adata_.obs['cell_type'] = adata.obs['cell_type']
adata_.obs['batch'] = adata.obs['batch']

In [15]:
%%R -i adata_
cite = as.Seurat(adata_, counts='X', data=NULL)
cite

R[write to console]: Warning:
R[write to console]:  Feature names cannot have underscores ('_'), replacing with dashes ('-')



An object of class Seurat 
87 features across 52117 samples within 1 assay 
Active assay: originalexp (87 features, 0 variable features)


# Integrate RNA batches

In [16]:
%%R
rna <- RenameAssays(rna, originalexp = 'RNA')
rna.list <- SplitObject(rna, split.by = "batch")
rna.list <- lapply(X = rna.list, FUN = SCTransform, variable.features.n = 2000)
features <- SelectIntegrationFeatures(object.list = rna.list, nfeatures = 2000)
rna.list <- PrepSCTIntegration(object.list = rna.list, anchor.features = features)

anchors <- FindIntegrationAnchors(object.list = rna.list, normalization.method = "SCT", 
    anchor.features = features)
integrated_rna <- IntegrateData(anchorset = anchors, normalization.method = "SCT")

integrated_rna <- RunPCA(integrated_rna)

R[write to console]: Renaming default assay from originalexp to RNA

R[write to console]: Calculating cell attributes from input UMI matrix: log_umi

R[write to console]: Variance stabilizing transformation of count matrix of size 3214 by 27740

R[write to console]: Model formula is y ~ log_umi

R[write to console]: Get Negative Binomial regression parameters per gene

R[write to console]: Using 2000 genes, 5000 cells



  |======================================================================| 100%


R[write to console]: There are 1 estimated thetas smaller than 1e-07 - will be set to 1e-07

R[write to console]: Found 125 outliers - those will be ignored in fitting/regularization step


R[write to console]: Second step: Get residuals using fitted parameters for 3214 genes



  |======================================================================| 100%


R[write to console]: Computing corrected count matrix for 3214 genes



...

R[write to console]: Finding all pairwise anchors



  |                                                  | 0 % ~calculating  

R[write to console]: Running CCA

R[write to console]: Merging objects

R[write to console]: Finding neighborhoods

R[write to console]: Finding anchors

R[write to console]: 	Found 53861 anchors

R[write to console]: Filtering anchors

R[write to console]: 	Retained 30903 anchors



  |++++++++++++++++++++++++++++++++++++++++++++++++++| 100% elapsed=03h 38m 44s


R[write to console]: Merging dataset 2 into 1

R[write to console]: Extracting anchors for merged samples

R[write to console]: Finding integration vectors

R[write to console]: Finding integration vector weights

R[write to console]: 0%   10   20   30   40   50   60   70   80   90   100%

R[write to console]: [----|----|----|----|----|----|----|----|----|----|

R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to con

# Integrate ADT batches

In [17]:
%%R
cite <- RenameAssays(cite, originalexp ='ADT')

cite.list <- SplitObject(cite, split.by = "batch")

cite.list <- lapply(X = cite.list, FUN = function(x) {
    VariableFeatures(x) <- rownames(x[["ADT"]])
    x <- NormalizeData(x, normalization.method = 'CLR', margin = 2, verbose=FALSE)
})

features <- SelectIntegrationFeatures(object.list = cite.list)

cite.list <- lapply(X = cite.list, FUN = function(x) {
    x <- ScaleData(x, features = features, verbose=FALSE)
    x <- RunPCA(x, features = features, reduction.name = "pca", verbose=FALSE)
})

anchors <- FindIntegrationAnchors(object.list = cite.list, reduction = "rpca", 
    dims = 1:30, verbose=FALSE)
integrated_adt <- IntegrateData(anchorset = anchors, dims = 1:30)

integrated_adt <- ScaleData(integrated_adt, verbose=FALSE)
integrated_adt <- RunPCA(integrated_adt, reduction.name = "apca", verbose=FALSE)

R[write to console]: Renaming default assay from originalexp to ADT

R[write to console]: Warning in irlba(A = t(x = object), nv = npcs, ...) :
R[write to console]: 
 
R[write to console]:  You're computing too large a percentage of total singular values, use a standard svd instead.

R[write to console]: Warning in irlba(A = t(x = object), nv = npcs, ...) :
R[write to console]: 
 
R[write to console]:  You're computing too large a percentage of total singular values, use a standard svd instead.



  |                                                  | 0 % ~calculating   |++++++++++++++++++++++++++++++++++++++++++++++++++| 100% elapsed=24s  


R[write to console]: Merging dataset 2 into 1

R[write to console]: Extracting anchors for merged samples

R[write to console]: Finding integration vectors

R[write to console]: Finding integration vector weights

R[write to console]: 0%   10   20   30   40   50   60   70   80   90   100%

R[write to console]: [----|----|----|----|----|----|----|----|----|----|

R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to con

# Extracting batch integrated data

In [18]:
%%R
#data preparation for UMINT\n",
rna_data <- as.matrix(GetAssayData(integrated_rna, slot = "scale.data", assay="integrated"))
write.csv(rna_data, 'E:/UMINTv2/Data/kotliarov50k/kotliarov50k_rna_scaled_batch_integrated.csv')

adt_data <- as.matrix(GetAssayData(integrated_adt, slot = "scale.data", assay="integrated"))
write.csv(adt_data, 'E:/UMINTv2/Data/kotliarov50k/kotliarov50k_adt_scaled_batch_integrated.csv')